Title:


Introduction:
Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
Clearly state the question you will try to answer with your project
Identify and describe the dataset that will be used to answer the question

Preliminary exploratory data analysis:
Demonstrate that the dataset can be read from the web into R 
Clean and wrangle your data into a tidy format
Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis. 

Dataset Origin: https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows

In [ ]:
### Run this cell before continuing. 
library(tidyverse)
library(repr)
library(rvest)
library(stringr)
options(repr.matrix.max.rows = 6)

In [ ]:
url <- "https://github.com/anh-dong/dsci-100-2023w1-group-33/blob/main/data/imdb_top_1000.csv?raw=true"
movies_raw <- read_csv(url) |>
    select(-Poster_Link, -Overview) |>
    filter(Released_Year > 1970)
# movies_raw
# selectors <- paste() asdf
movies_raw

In [ ]:
movies <- movies_raw |>
    select(Released_Year, Meta_score, Gross, Runtime, IMDB_Rating) |>
    na.omit() |>
    mutate(Runtime = strtoi(Runtime %>% str_replace(" min", ""))) |>
    group_by(Released_Year) |>
    summarize(ave_Gross = mean(Gross), ave_runtime = mean(Runtime)*60, ave_score = mean(Meta_score), ave_IMDB_Rating = mean(IMDB_Rating)) |>
    arrange(Released_Year)
# movies

movies_plot <- movies |>
    ggplot(aes(Released_Year, ave_Gross)) +
        geom_line() +
        xlab("Year") +
        ylab("Gross (USD)") +
        ggtitle("Average Yearly Gross for Movies over time")
movies_plot

movies_plot_2 <- movies |>
    ggplot(aes(Released_Year, ave_runtime)) +
        geom_line() +
        xlab("Year") +
        ylab("Average Runtime (hours)") +
        ggtitle("Average Runtime for Movies over time")
movies_plot_2

movies_plot_3 <- movies |>
    ggplot(aes(Released_Year, ave_IMDB_Rating)) +
        geom_line() +
        xlab("Year") +
        ylab("Average IMDB Rating") +
        ggtitle("Average IMDB Rating for Movies over time")
movies_plot_3

In [ ]:
movies2 <- movies_raw |>
    select(Meta_score, Gross, Runtime, IMDB_Rating) |>
    na.omit() |>
    mutate(Runtime = strtoi(Runtime %>% str_replace(" min", ""))) |>
    group_by(Meta_score) |>
    summarize(ave_Gross = mean(Gross), ave_runtime = mean(Runtime)*60, ave_IMDB_Rating = mean(IMDB_Rating))

movies_plot_5 <- movies2 |>
    ggplot(aes(Meta_score, ave_Gross)) +
        geom_point() +
        xlab("Meta Score") +
        ylab("Ave Gross (USD)") +
        ggtitle("Meta Score vs Average Gross")
movies_plot_5

movies_plot_6 <- movies2 |>
    ggplot(aes(Meta_score, ave_runtime)) +
        geom_point() +
        xlab("Meta Score") +
        ylab("Ave Runtime (Hours)") +
        ggtitle("Meta Score vs Average Runtime")
movies_plot_6

movies_plot_7 <- movies2 |>
    ggplot(aes(Meta_score, ave_IMDB_Rating)) +
        geom_point() +
        xlab("Meta Score") +
        ylab("Ave IMDB Rating") +
        ggtitle("Meta Score vs Average IMDB Rating")
movies_plot_7

In [ ]:
movies3 <- movies_raw |>
    select(Meta_score, Gross, Runtime, IMDB_Rating) |>
    na.omit() |>
    mutate(Runtime = strtoi(Runtime %>% str_replace(" min", "")))

movies_plot_8 <- movies3 |>
    ggplot(aes(Meta_score, IMDB_Rating)) +
        geom_line() +
        xlab("Year") +
        ylab("Runtime (hours)") +
        ggtitle("Movie Length for Movies over time")
movies_plot_8

Methods:


Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
Describe at least one way that you will visualize the results

Expected outcomes and significance:


What do you expect to find?
What impact could such findings have?
What future questions could this lead to?